In [2]:
import pandas as pd
import glob
from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

In [11]:
# 폴더 내의 모든 CSV 파일을 읽어와 하나의 데이터 프레임으로 합치기
path = './data'  # CSV 파일이 저장된 폴더 경로
all_files = glob.glob(path + "/*.csv")


In [19]:
li = []
for filename in all_files:
    with open(filename, 'r') as temp_f:
        col_count = len(temp_f.readline().split(','))
    column_names = list(range(col_count))
    df = pd.read_csv(filename, index_col=None, names=column_names)
    li.append(df)

data = pd.concat(li, axis=0, ignore_index=True)

In [20]:
data

,0,1,2,3,4,5,6,7,8,9,...,48,49,50,51,52,53,54,55,56,57
0,4,1,856,275,1094,919,968.77844,985.80020,956.32117,1014.98170,...,0.982044,0.972097,0.969531,0.998568,0.998549,0.994824,0.994122,0.977357,0.975410,normal
1,4,0,1154,418,1368,934,1256.39000,1275.60830,1248.66170,1310.52120,...,0.956926,0.967062,0.944040,0.996357,0.995411,0.991412,0.988322,0.974259,0.968777,normal
2,5,1,856,276,1094,920,968.20240,985.34924,956.16504,1014.60840,...,0.981337,0.971873,0.968387,0.998535,0.998497,0.994722,0.993935,0.977015,0.974849,normal
3,5,0,1159,421,1360,931,1257.05640,1275.63180,1250.74170,1311.08360,...,0.953789,0.963013,0.934785,0.997092,0.996352,0.993481,0.991186,0.979521,0.975355,normal
4,6,1,858,276,1095,920,968.26807,985.31635,955.66970,1014.89820,...,0.976096,0.965058,0.958567,0.998208,0.998081,0.993554,0.992557,0.973141,0.970821,normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
443669,3596,486,683,153,1132,1073,1093.34970,1030.06300,1080.35380,873.09570,...,0.880025,0.094931,0.387825,0.947078,0.977476,0.680375,0.811679,0.344397,0.414159,a01
443670,3597,486,683,154,1133,1073,1095.41040,1032.80690,1081.61940,877.06530,...,0.886107,0.104240,0.411271,0.947411,0.977452,0.673676,0.806252,0.332119,0.399815,a01
443671,3598,486,683,155,1134,1072,1096.11490,1033.11020,1082.54530,876.41833,...,0.879471,0.102184,0.396786,0.946527,0.976559,0.673870,0.803527,0.335096,0.399575,a01
443672,3599,486,683,155,1135,1072,1097.35080,1033.68120,1083.36330,876.08310,...,0.865792,0.105633,0.378222,0.941860,0.972937,0.660247,0.786126,0.320169,0.373696,a01


In [21]:
# 데이터 전처리
y = data.iloc[:, -1].apply(lambda x: 1 if x == 'a01' else 0).values
X = data.iloc[:, 6:40].values
# Drop NaN

# 데이터 정규화 (1920x1080 크기를 기준으로 상대좌표로 변환)
X_normalized = X.copy()
X_normalized[:, 0::2] = X[:, 0::2] / 1920
X_normalized[:, 1::2] = X[:, 1::2] / 1080

In [22]:
print(y)

[0 0 0 ... 1 1 1]


In [27]:
# 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(X_normalized, y, test_size=0.2, random_state=42)

# SVM 모델 생성 및 학습
model = svm.SVC()
model.fit(X_train, y_train)

# 테스트 데이터에 대한 예측 수행
y_pred = model.predict(X_test)

TypeError: __init__() got an unexpected keyword argument 'n_jobs'

In [24]:
# 성능 평가
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)

In [25]:
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)
print("Confusion Matrix:\n", conf_matrix)

Accuracy: 0.876429819124359
Precision: 0.9014859658778206
Recall: 0.4984025559105431
F1 Score: 0.6419124130498678
Confusion Matrix:
 [[67942  1074]
 [ 9891  9828]]


In [26]:
from sklearn.externals import joblib

# Save the model
joblib.dump(model, 'model.pkl')


c:\Users\content\anaconda3\envs\yolov8_env\lib\site-packages\sklearn\externals\joblib\__init__.py:15: FutureWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=FutureWarning)


['model.pkl']